# Information Gain
### While reading about decision trees in the fist chapter of <a href="http://ciml.info/">A Course in Machine Learning</a> I wanted to understand exactly what is "information gain" and after reading <a href="https://en.wikipedia.org/wiki/Information_gain_in_decision_trees">Wikipedia article</a> and <a href="https://towardsdatascience.com/decision-trees-for-classification-id3-algorithm-explained-89df76e72df1">a Medium story</a> found the matter still unclear.
### Fortunately Wikipedia referenced <a href="https://www.numpyninja.com/post/what-is-entropy-and-information-gain-how-are-they-used-to-construct-decision-trees">this great article on Numpy Ninja</a> and now I understand IG.

In [ ]:
# Data from the article's example

data = [(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), 
        (1, 0), (1, 0), (1, 0), (1, 0),
        (0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]

dlen = len(data)

ysum = sum(y for _, y in data)
print(ysum)

### Data is a list of (x, y) where y is our target variable. It has 2 values - 1 and 0
### Now let's calculate entropy. It is a negative sum of target value frequences multiplied by frequency logarithm

In [ ]:
from math import log

def entropy(oclist):
    res = 0
    total = sum(oclist)
    
    for o in oclist:
        f = o/total
        res -= (f*log(f, 2))
    
    return res

In [ ]:
parent_entropy = entropy([ysum, dlen - ysum]) 
print(parent_entropy)

### Let's split our data on x

In [ ]:
data0 = [y for x, y in data if x==0]
print(data0)

In [ ]:
data1 = [y for x, y in data if x==1]
print(data1)

In [ ]:
ysum0 = sum(data0)
dlen0 = len(data0)
child0_entropy = entropy([ysum0, dlen0 - ysum0])

ysum1 = sum(data1)
dlen1 = len(data1)
child1_entropy = entropy([ysum1, dlen1 - ysum1])

print('child0_entropy = {}'.format(child0_entropy))
print('child1_entropy = {}'.format(child1_entropy))      

### Weighted average entropy of children

In [ ]:
wae = (dlen0/dlen) * child0_entropy + (dlen1/dlen) * child1_entropy 
print(wae)

### And at last: ***Information Gain***

In [ ]:
ig = parent_entropy - wae
print(ig)

# Full-fledged decision tree: 
## binary, greedy, using weighted average entropy of children

In [ ]:
import pandas as pd

def next_split(df, y_name):
    'Finds index and threshold to split a df for the max information gain on y index'

    df_len = len(df.index)
    wae = float('inf')
    ename = None
    ethreshold = None
    edfl = None
    edfr = None

    x_names = (name for name, _ in df.iteritems() if name != y_name)    
    for name in x_names:
        vals = sorted(df[name].value_counts().index)
        for vi in range(1, len(vals)):
            dfl = df[df[name] < vals[vi]]
            dfr = df.drop(dfl.index)
            local_wae = (len(dfl.index) / df_len) * entropy(dfl[y_name].value_counts()) \
                      + (len(dfr.index) / df_len) * entropy(dfr[y_name].value_counts())
            if wae > local_wae:
                wae = local_wae
                ename = name
                ethreshold = vals[vi]
                edfl = dfl.drop(columns = [name])
                edfr = dfr.drop(columns = [name])

    if ename:
        return (ename, ethreshold, edfl, edfr)
        
    return (None, None, None, None)    
    
class Leaf:
    def __init__(self, val):
        self.value = val
        
    def label(self, row):
        return self.value
        
class Node(Leaf):
    def __init__(self, name, threshold):
        super().__init__(None)
        self.name = name
        self.threshold = threshold
        self.l = None
        self.r = None
        
    def label(self, row):
        if row[self.name] < self.threshold:
            return self.l.label(row)
        else:
            return self.r.label(row)

def build_tree(df, y_name):
    vc = df[y_name].value_counts()
    if len(vc) > 1:
        sname, sthreshold, sdfl, sdfr = next_split(df, y_name)
        if (sname):
            ret = Node(sname, sthreshold)
            ret.l = build_tree(sdfl, y_name)
            ret.r = build_tree(sdfr, y_name)
            return ret
    return Leaf(df[y_name].mean())
    

In [ ]:
# Cervical Cancer Behavior Risk Data Set
cc_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00537/sobar-72.csv')


In [ ]:
cc_data.info()

In [ ]:
y_name = 'ca_cervix'

# split to train and test using groupby in order to have roughly the same y-entropy in original, train and test sets
df_train = cc_data.groupby(y_name).sample(frac=0.7, random_state = 9)
df_test = cc_data.drop(df_train.index)


In [ ]:
entropy(cc_data[y_name].value_counts()), entropy(df_train[y_name].value_counts()), entropy(df_test[y_name].value_counts())

In [ ]:
root = build_tree(df_train, y_name)

In [ ]:
n_t = 0
n_f = 0

for _, row in df_test.iterrows():    
    res = root.label(row)
    if res == row[y_name]:
        n_t += 1
    else:
        n_f += 1

print('{} samples labeled correctly, {} samples labeled wrongly, accuracy = {}'.format(n_t, n_f, n_t/(n_t + n_f)))